#  <center> Neural Networks and Fuzzy Control EEE1007 </center>

## <center> Face Recognition using Python and Keras (TensorFlow) </center>
<br>

<b> Rishu Sinha 15BEE0114  </b><br>
<b> Anupam Bisht 15BEE0139 <br> </b>
<b> Siddharth Bhargava 15BCE0628 </b>
<br><br>
<em>Professor Mathew M. Noel </em><br>
**Slot**: G1
<br>



## Introduction:

In [30]:
import cv2
import numpy as np
import os
from skimage import io
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

### Import the dataset:

We are using the yalefaces dataset, collected from the [this link](http://vismod.media.mit.edu/vismod/classes/mas622-00/datasets/YALE.tar.gz "Yale Face Database Download"). <br>
TO know more about the yale faces database, you can read up at [this link](http://vision.ucsd.edu/content/yale-face-database "Yale Face Database").<br>
To know more about face recognition databases, you can read up at [this link](http://www.face-rec.org/databases/ "Face Recognition Databases"). A lot of different face databases are available here.

In [32]:
DatasetPath = []
for i in os.listdir("yalefaces"):
    DatasetPath.append(os.path.join("yalefaces", i))

#### Store the dataset in a list along with their respective labels

We use skimage library to read the images from the folder and then convert them to their grayscale version. The labels have been extracted from the filepath of each image using os library.

In [33]:
imageData = []
imageLabels = []

In [34]:
for i in DatasetPath:
    imgRead = io.imread(i,as_grey=True)
    imageData.append(imgRead)
    
    labelRead = int(os.path.split(i)[1].split(".")[0].replace("subject", "")) - 1
    imageLabels.append(labelRead)

### use OpenCV for pre-processing of the image dataset:

Here, we are going to scale the image dataset to a resolution of _150 x 150_. We use the most popular face detection tool provided by OpenCV, **haarcascade_frontalface_default.xml**. This classifier is used for effective face detection and the classifier has been trained to extract the useful features from the dataset. 

In [35]:
faceDetectClassifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [36]:
imageDataFin = []
for i in imageData:
    facePoints = faceDetectClassifier.detectMultiScale(i)
    x,y = facePoints[0][:2]
    cropped = i[y: y + 150, x: x + 150]
    imageDataFin.append(cropped)

In [37]:
c = np.array(imageDataFin)
print(c.shape)

(165, 150, 150)


### Model Set Construction

We are going to use the in-built function, *train_test_split*, to split the dataset into training set and testing set as per our defined ratio. The shapes of each are shown here: <br>
    <center> <b>X_train:</b> (no_of_examples_in_training_set, width x height) </center>
    <center> <b>X_test:</b> (no_of_examples_in_test_set, width x height) </center><br>

As per the dataset, we have 15 different subjects in the dataset. Thus the number of classes defined are 15. We can add people or remove as and when we want. <br>
Using _np.utils_ we have have defined the classes of each image collected from **y_train**.<br>
**X_train** and **X_test** are reshaped into the desired number to ensure there's no discrepancy. We set the type as 'float32' and scale it down to facilitate in the processing step.

>#### Why do we need to reshape and scale the training set?
    


In [11]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin),np.array(imageLabels), train_size=0.9, test_size=0.1, random_state = 20)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (148, 150, 150)
X_test shape: (17, 150, 150)


In [14]:
nb_classes = 15
y_train = np.array(y_train) 
y_test = np.array(y_test)

In [15]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [16]:
X_train = X_train.reshape((148, 150*150))
X_test = X_test.reshape((17, 150*150))

In [17]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [18]:
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (148, 22500)
Testing matrix shape (17, 22500)


### Forming the neural network using Keras

We are using a **3-layer neural network** with self-defined hidden units and activation function. 

For the first hidden layer, we are using 512 nodes and the activation function used is _relu_. The output of the first hidden layer gets fed into the second hidden layer which has same number of hidden units and activation function.  

The third layer or the output layer has the number of nodes equal to the _number of classes_ defined in the dataset. The third layer uses **softmax** as a activation function as the dataset is a multi-class classification problem. 


>#### What is model and Sequential()?
>

We have used **dropout** as the regularization technique.
>#### What is regularization and dropout?

In [19]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               11520512  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                7695      
__________

We can easily calculate the number of parameters for each of the layers. 
> **First Layer**: weights + bias = 22500 x 512 + 512 = 11520512 <br>
> **Second Layer**: weights + bias = 512 x 512 + 512 = 262656 <br>
> **Third Layer**: weights + bias = 512 x 15 + 15 = 7695 <br>

### Running the Model

We have hand-picked the hyper-parameters like the _batch-size_ and  _number of epochs_ based on the most optinum combination. The optimizer selected here is **AdamOptimizer**. <br>
We define the loss function as 

In [21]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
 

In [22]:
model.fit(X_train, Y_train, batch_size=64, epochs=50, verbose=1, validation_data=(X_test, Y_test))


Train on 148 samples, validate on 17 samples
Epoch 1/50
148/148 [==============================] - 2s 12ms/step - loss: 7.3626 - acc: 0.0743 - val_loss: 10.5325 - val_acc: 0.1176
Epoch 2/50
148/148 [==============================] - 1s 8ms/step - loss: 12.5365 - acc: 0.1081 - val_loss: 11.1299 - val_acc: 0.1765
Epoch 3/50
148/148 [==============================] - 1s 7ms/step - loss: 12.2671 - acc: 0.1216 - val_loss: 9.0499 - val_acc: 0.2353
Epoch 4/50
148/148 [==============================] - 1s 7ms/step - loss: 11.8310 - acc: 0.1757 - val_loss: 9.0031 - val_acc: 0.1765
Epoch 5/50
148/148 [==============================] - 1s 7ms/step - loss: 12.1741 - acc: 0.1689 - val_loss: 7.1914 - val_acc: 0.3529
Epoch 6/50
148/148 [==============================] - 1s 7ms/step - loss: 12.2930 - acc: 0.1486 - val_loss: 8.1586 - val_acc: 0.3529
Epoch 7/50
148/148 [==============================] - 1s 7ms/step - loss: 11.6429 - acc: 0.2297 - val_loss: 10.0574 - val_acc: 0.2941
Epoch 8/50
148/148 [=

### Evaluating the performance of the Network

In [23]:
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)

In [24]:
loss

0.21521124243736267

In [25]:
accuracy

0.94117647409439087

In [26]:
predicted_classes= model.predict_classes(X_test)

17/17 [==============================] - 0s 3ms/step


In [27]:
correct_classified_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_classified_indices = np.nonzero(predicted_classes != y_test)[0]

In [28]:
correct_classified_indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16], dtype=int64)

In [29]:
incorrect_classified_indices

array([0], dtype=int64)

### Conclusion